# Use Folium to Display Maps

In [2]:
import folium
map_osm = folium.Map(location=[45.5236, -122.6750])
map_osm.save('osm.html')

# Bike Routes in Dublin

In [1]:
import pandas as pd
import folium
location = "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/2dec86ed-76ed-47a3-ae28-646db5c5b965/download/dublin.csv"
bike_station_locations = pd.read_csv(location)
bike_station_locations = bike_station_locations[["Latitude", "Longitude", "Name"]]
map = folium.Map(location=[bike_station_locations.Latitude.mean(), bike_station_locations.Longitude.mean()], zoom_start=14, control_scale=True)
for index, location_info in bike_station_locations.iterrows():
    folium.Marker([location_info["Latitude"], location_info["Longitude"]], popup=location_info["Name"]).add_to(map)

# save to html file
map.save("index.html")

In [2]:
import pandas as pd
import folium
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()



Successfully saved authorization token.


In [24]:
# Define a method for displaying Earth Engine image tiles to folium map.

# Use LSIB Large Scale International Boundaries to select Region
import folium

# Define the center of our map.
lat, lon = 50.77, 0.855

myMap = folium.Map(location=[lat, lon], zoom_start=10)
worldcountries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');
filterCountry = ee.Filter.eq('country_na', 'United Kingdom');
country = worldcountries.filter(filterCountry);

# Define the visualization parameters.

# Import the MODIS land surface temperature collection.
# LST = Land Surface Temperature
lst = ee.ImageCollection('MODIS/006/MOD11A1')

landsat = ee.ImageCollection(
    'LANDSAT/LC08/C01/T1').filterDate('2016-01-01', '2017-01-01').filterBounds(country)
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat,
    'asFloat': True
})

# Fetch an elevation model.
# dem = ee.Image('USGS/SRTMGL1_003')

rgbVis = {'bands': ['B3', 'B4'], 'min': 0, 'max': 0.3}
nirVis = {'bands': ['B5', 'B4'], 'min': 0, 'max': [0.5, 0.3]}


# This is needed in python to add layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


# Initial date of interest (inclusive).
i_date = '2017-01-01'
# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)
lc = ee.ImageCollection('MODIS/006/MCD12Q1')
lc_img = lc.select('LC_Type1').filterDate(i_date).first()

# Set visualization parameters for land cover.
lc_vis_params = {
    'min': 1,'max': 17,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']
}

# Create a map.
lat, lon = 45.77, 4.855
myMap = folium.Map(location=[lat, lon], zoom_start=7)

# Add the land cover to the map object.
myMap.add_ee_layer(lc_img, lc_vis_params, 'Land Cover')
# Add the elevation model to the map object.
myMap.add_ee_layer(dem, visParams, 'DEM')
myMap.add_ee_layer(composite, rgbVis, 'RGB')
myMap.add_ee_layer(composite, nirVis, 'false Color')
# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)
myMap.save("myMap.html")